In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.6 MB/s 
     |████████████████████████████████| 86 kB 3.0 MB/s 
     |████████████████████████████████| 6.6 MB 30.5 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling,AutoModelForSequenceClassification
from transformers import BertTokenizer, LineByLineTextDataset, TrainingArguments, Trainer, XLMRobertaTokenizer, AutoTokenizer
from transformers import BertForSequenceClassification, AdamW, BertTokenizer, BertModel

In [3]:
import pandas as pd
import numpy as np

In [4]:
checkpoint = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(checkpoint, do_lower_case=True)
model = BertModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
df = pd.DataFrame()
# for i in range(1, 32):
#     path = '/content/drive/MyDrive/guidao/data_3_4/score' + str(i) + '_ls_df.csv'
#     dtt = pd.read_csv(path)
#     df = pd.concat([df, dtt])

#for i in range(1, 16):
    #path = '/content/drive/MyDrive/guidao/data/score' + str(i) + '_df.csv'
    #dtt = pd.read_csv(path)
    #df = pd.concat([df, dtt])
dtt=pd.read_csv("/content/drive/MyDrive/第三轮主动学习数据/al_0608_2.csv")
df = pd.concat([df, dtt])
df

,Unnamed: 0,nopunc,score,ids2,label,cluster_db,Phrase,sentiment_label,pred_pro
0,18035,本报 评论员 日本首相 岸信 介的 东南亚 五国 之行 从 月 日 抵达 仰光 到 月 日 ...,"[-0.0038761309260749496, -0.11965509088785244,...","[0.0005283885, 0.00085630326, 0.0008567112, 0....",0,0,"['本报', '评论员', '日本首相', '岸信', '介的', '东南亚', '五国',...","[1, 1, 0, 0, 0]","[0.8190277814865112, 0.6120412349700928, 0.963..."
1,3571,死亡 人数 仍 居高不下 重 特大事故 尚未 有效 遏制 本报 北京 月 日 讯 记者 彭嘉...,"[0.0046436703015258415, -0.12677482132145174, ...","[0.00051479205, 0.0008267846, 0.0007416491, 0....",0,0,"['死亡', '人数', '仍', '居高不下', '重', '特大事故', '尚未', '...","[1, 1]","[0.8987365365028381, 0.6969270706176758]"
2,12326,新华社 北京 月 日电 日本自民党 总务 会长 伊东 正义 先生 作为 日本首相 特使 预定...,"[0.3578358898089807, 0.08161251164610346, 0.11...","[0.0004722392, 0.00072869874, 0.00058751204, 0...",0,0,"['新华社', '北京', '月', '日电', '日本自民党', '总务', '会长', ...",[0],[0.7675070762634277]
3,29706,本报 宁波 月 日电 记者 陆娅楠 月 日 杉杉 集团 有限公司 与 日本 伊藤忠 商事 株...,"[-0.303011174543276, 0.0032854388120253653, 0....","[0.00045990184, 0.0007084661, 0.0006132199, 0....",0,0,"['本报', '宁波', '月', '日电', '记者', '陆娅楠', '月', '日',...",[1],[0.6792555451393127]
4,7697,新华社 一九七五年 一月 十五日 讯 据 日本 东方 通讯社 报道 一九七四年 日本 企业倒...,"[-0.0059179321907659906, -0.0731447363179498, ...","[0.0004908887, 0.0007480143, 0.000604546, 0.00...",0,0,"['新华社', '一九七五年', '一月', '十五日', '讯', '据', '日本', ...","[0, 0]","[0.8662652969360352, 0.9438633322715759]"
5,2437,本报 东京 月 日电 记者 于青 报道 日本政府 内阁会议 今天 决定 向 中国 南部 水灾...,"[-0.02468244892408373, 0.16543349252372455, 0....","[0.00046329183, 0.00070480444, 0.0005240025, 0...",0,0,"['本报', '东京', '月', '日电', '记者', '于青', '报道', '日本政...",[1],[0.7693524360656738]
6,13868,本报讯 记者 李丽辉 报道 日前 中国保监会 主席 吴定富 宣布 中国保监会 批准 英国 标...,"[0.056018661112242046, 0.027020653065623686, -...","[0.0004801159, 0.00074307574, 0.000648965, 0.0...",0,0,"['本报讯', '记者', '李丽辉', '报道', '日前', '中国保监会', '主席'...",[1],[0.7656781673431396]
7,18301,为期 两天 的 中国 东盟 文化 部长会议 与 东盟 中日韩 文化部长 系列 会议 于 日 ...,"[0.2538221689732428, -0.12466527668357397, -0....","[0.0004300173, 0.0006546416, 0.00044636868, 0....",0,0,"['为期', '两天', '的', '中国', '东盟', '文化', '部长会议', '与...",[1],[0.5380568504333496]
8,6051,我 华侨 政策 不容 诋毁 越南 排华 是 为了 反华 苏越 合谋 反华 是因为 中国 坚决...,"[0.04909594366811509, -0.25454754978558125, -0...","[0.0005256694, 0.00087940495, 0.0010792044, 0....",0,0,"['我', '华侨', '政策', '不容', '诋毁', '越南', '排华', '是',...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.5756285190582275, 0.7522038817405701, 0.949..."
9,5333,从 二三百 美元 达到 一千美元 要 多少 年 问 编辑 同志 我们 想 了解 一下 世界 ...,"[0.005414972940424628, 0.006805516641449858, 0...","[0.00048636642, 0.00074270775, 0.0006064196, 0...",5,0,"['从', '二三百', '美元', '达到', '一千美元', '要', '多少', '年...","[1, 1, 1]","[0.5769215822219849, 0.8073752522468567, 0.929..."


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df.index = range(len(df))
df

,Unnamed: 0,nopunc,score,ids2,label,cluster_db,Phrase,sentiment_label,pred_pro
0,18035,本报 评论员 日本首相 岸信 介的 东南亚 五国 之行 从 月 日 抵达 仰光 到 月 日 ...,"[-0.0038761309260749496, -0.11965509088785244,...","[0.0005283885, 0.00085630326, 0.0008567112, 0....",0,0,"['本报', '评论员', '日本首相', '岸信', '介的', '东南亚', '五国',...","[1, 1, 0, 0, 0]","[0.8190277814865112, 0.6120412349700928, 0.963..."
1,3571,死亡 人数 仍 居高不下 重 特大事故 尚未 有效 遏制 本报 北京 月 日 讯 记者 彭嘉...,"[0.0046436703015258415, -0.12677482132145174, ...","[0.00051479205, 0.0008267846, 0.0007416491, 0....",0,0,"['死亡', '人数', '仍', '居高不下', '重', '特大事故', '尚未', '...","[1, 1]","[0.8987365365028381, 0.6969270706176758]"
2,12326,新华社 北京 月 日电 日本自民党 总务 会长 伊东 正义 先生 作为 日本首相 特使 预定...,"[0.3578358898089807, 0.08161251164610346, 0.11...","[0.0004722392, 0.00072869874, 0.00058751204, 0...",0,0,"['新华社', '北京', '月', '日电', '日本自民党', '总务', '会长', ...",[0],[0.7675070762634277]
3,29706,本报 宁波 月 日电 记者 陆娅楠 月 日 杉杉 集团 有限公司 与 日本 伊藤忠 商事 株...,"[-0.303011174543276, 0.0032854388120253653, 0....","[0.00045990184, 0.0007084661, 0.0006132199, 0....",0,0,"['本报', '宁波', '月', '日电', '记者', '陆娅楠', '月', '日',...",[1],[0.6792555451393127]
4,7697,新华社 一九七五年 一月 十五日 讯 据 日本 东方 通讯社 报道 一九七四年 日本 企业倒...,"[-0.0059179321907659906, -0.0731447363179498, ...","[0.0004908887, 0.0007480143, 0.000604546, 0.00...",0,0,"['新华社', '一九七五年', '一月', '十五日', '讯', '据', '日本', ...","[0, 0]","[0.8662652969360352, 0.9438633322715759]"
5,2437,本报 东京 月 日电 记者 于青 报道 日本政府 内阁会议 今天 决定 向 中国 南部 水灾...,"[-0.02468244892408373, 0.16543349252372455, 0....","[0.00046329183, 0.00070480444, 0.0005240025, 0...",0,0,"['本报', '东京', '月', '日电', '记者', '于青', '报道', '日本政...",[1],[0.7693524360656738]
6,13868,本报讯 记者 李丽辉 报道 日前 中国保监会 主席 吴定富 宣布 中国保监会 批准 英国 标...,"[0.056018661112242046, 0.027020653065623686, -...","[0.0004801159, 0.00074307574, 0.000648965, 0.0...",0,0,"['本报讯', '记者', '李丽辉', '报道', '日前', '中国保监会', '主席'...",[1],[0.7656781673431396]
7,18301,为期 两天 的 中国 东盟 文化 部长会议 与 东盟 中日韩 文化部长 系列 会议 于 日 ...,"[0.2538221689732428, -0.12466527668357397, -0....","[0.0004300173, 0.0006546416, 0.00044636868, 0....",0,0,"['为期', '两天', '的', '中国', '东盟', '文化', '部长会议', '与...",[1],[0.5380568504333496]
8,6051,我 华侨 政策 不容 诋毁 越南 排华 是 为了 反华 苏越 合谋 反华 是因为 中国 坚决...,"[0.04909594366811509, -0.25454754978558125, -0...","[0.0005256694, 0.00087940495, 0.0010792044, 0....",0,0,"['我', '华侨', '政策', '不容', '诋毁', '越南', '排华', '是',...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.5756285190582275, 0.7522038817405701, 0.949..."
9,5333,从 二三百 美元 达到 一千美元 要 多少 年 问 编辑 同志 我们 想 了解 一下 世界 ...,"[0.005414972940424628, 0.006805516641449858, 0...","[0.00048636642, 0.00074270775, 0.0006064196, 0...",5,0,"['从', '二三百', '美元', '达到', '一千美元', '要', '多少', '年...","[1, 1, 1]","[0.5769215822219849, 0.8073752522468567, 0.929..."


In [8]:
def txt2embemding(txt):
    if len(txt) > 500:
        txt = txt[:500]
    encoded_input = tokenizer(txt, return_tensors='pt', padding="max_length", max_length=500)
#     encoded_input = tokenizer(txt, return_tensors='pt')
    output = model(**encoded_input)
    output = F.softmax(output.pooler_output[0])
    embeding = output.detach().numpy()
    return embeding

In [9]:
import time

In [10]:
start = time.perf_counter()

df['ids'] = df.nopunc.apply(txt2embemding)

end = time.perf_counter()
print('Running time: %s Seconds' % (end - start))
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


Running time: 96.34702801900005 Seconds


,Unnamed: 0,nopunc,score,ids2,label,cluster_db,Phrase,sentiment_label,pred_pro,ids
0,18035,本报 评论员 日本首相 岸信 介的 东南亚 五国 之行 从 月 日 抵达 仰光 到 月 日 ...,"[-0.0038761309260749496, -0.11965509088785244,...","[0.0005283885, 0.00085630326, 0.0008567112, 0....",0,0,"['本报', '评论员', '日本首相', '岸信', '介的', '东南亚', '五国',...","[1, 1, 0, 0, 0]","[0.8190277814865112, 0.6120412349700928, 0.963...","[0.0005283885, 0.00085630326, 0.0008567112, 0...."
1,3571,死亡 人数 仍 居高不下 重 特大事故 尚未 有效 遏制 本报 北京 月 日 讯 记者 彭嘉...,"[0.0046436703015258415, -0.12677482132145174, ...","[0.00051479205, 0.0008267846, 0.0007416491, 0....",0,0,"['死亡', '人数', '仍', '居高不下', '重', '特大事故', '尚未', '...","[1, 1]","[0.8987365365028381, 0.6969270706176758]","[0.00051479205, 0.0008267846, 0.0007416491, 0...."
2,12326,新华社 北京 月 日电 日本自民党 总务 会长 伊东 正义 先生 作为 日本首相 特使 预定...,"[0.3578358898089807, 0.08161251164610346, 0.11...","[0.0004722392, 0.00072869874, 0.00058751204, 0...",0,0,"['新华社', '北京', '月', '日电', '日本自民党', '总务', '会长', ...",[0],[0.7675070762634277],"[0.0004722392, 0.00072869874, 0.00058751204, 0..."
3,29706,本报 宁波 月 日电 记者 陆娅楠 月 日 杉杉 集团 有限公司 与 日本 伊藤忠 商事 株...,"[-0.303011174543276, 0.0032854388120253653, 0....","[0.00045990184, 0.0007084661, 0.0006132199, 0....",0,0,"['本报', '宁波', '月', '日电', '记者', '陆娅楠', '月', '日',...",[1],[0.6792555451393127],"[0.00045990184, 0.0007084661, 0.0006132199, 0...."
4,7697,新华社 一九七五年 一月 十五日 讯 据 日本 东方 通讯社 报道 一九七四年 日本 企业倒...,"[-0.0059179321907659906, -0.0731447363179498, ...","[0.0004908887, 0.0007480143, 0.000604546, 0.00...",0,0,"['新华社', '一九七五年', '一月', '十五日', '讯', '据', '日本', ...","[0, 0]","[0.8662652969360352, 0.9438633322715759]","[0.0004908887, 0.0007480143, 0.000604546, 0.00..."
5,2437,本报 东京 月 日电 记者 于青 报道 日本政府 内阁会议 今天 决定 向 中国 南部 水灾...,"[-0.02468244892408373, 0.16543349252372455, 0....","[0.00046329183, 0.00070480444, 0.0005240025, 0...",0,0,"['本报', '东京', '月', '日电', '记者', '于青', '报道', '日本政...",[1],[0.7693524360656738],"[0.00046329183, 0.00070480444, 0.0005240025, 0..."
6,13868,本报讯 记者 李丽辉 报道 日前 中国保监会 主席 吴定富 宣布 中国保监会 批准 英国 标...,"[0.056018661112242046, 0.027020653065623686, -...","[0.0004801159, 0.00074307574, 0.000648965, 0.0...",0,0,"['本报讯', '记者', '李丽辉', '报道', '日前', '中国保监会', '主席'...",[1],[0.7656781673431396],"[0.0004801159, 0.00074307574, 0.000648965, 0.0..."
7,18301,为期 两天 的 中国 东盟 文化 部长会议 与 东盟 中日韩 文化部长 系列 会议 于 日 ...,"[0.2538221689732428, -0.12466527668357397, -0....","[0.0004300173, 0.0006546416, 0.00044636868, 0....",0,0,"['为期', '两天', '的', '中国', '东盟', '文化', '部长会议', '与...",[1],[0.5380568504333496],"[0.0004300173, 0.0006546416, 0.00044636868, 0...."
8,6051,我 华侨 政策 不容 诋毁 越南 排华 是 为了 反华 苏越 合谋 反华 是因为 中国 坚决...,"[0.04909594366811509, -0.25454754978558125, -0...","[0.0005256694, 0.00087940495, 0.0010792044, 0....",0,0,"['我', '华侨', '政策', '不容', '诋毁', '越南', '排华', '是',...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.5756285190582275, 0.7522038817405701, 0.949...","[0.0005256694, 0.00087940495, 0.0010792044, 0...."
9,5333,从 二三百 美元 达到 一千美元 要 多少 年 问 编辑 同志 我们 想 了解 一下 世界 ...,"[0.005414972940424628, 0.006805516641449858, 0...","[0.00048636642, 0.00074270775, 0.0006064196, 0...",5,0,"['从', '二三百', '美元', '达到', '一千美元', '要', '多少', '年...","[1, 1, 1]","[0.5769215822219849, 0.8073752522468567, 0.929...","[0.00048636642, 0.00074270775, 0.0006064196, 0..."


In [11]:
def change2list(ls):
    return list(ls)
df['ids2'] = df.ids.apply(change2list)
df

,Unnamed: 0,nopunc,score,ids2,label,cluster_db,Phrase,sentiment_label,pred_pro,ids
0,18035,本报 评论员 日本首相 岸信 介的 东南亚 五国 之行 从 月 日 抵达 仰光 到 月 日 ...,"[-0.0038761309260749496, -0.11965509088785244,...","[0.0005283885, 0.00085630326, 0.0008567112, 0....",0,0,"['本报', '评论员', '日本首相', '岸信', '介的', '东南亚', '五国',...","[1, 1, 0, 0, 0]","[0.8190277814865112, 0.6120412349700928, 0.963...","[0.0005283885, 0.00085630326, 0.0008567112, 0...."
1,3571,死亡 人数 仍 居高不下 重 特大事故 尚未 有效 遏制 本报 北京 月 日 讯 记者 彭嘉...,"[0.0046436703015258415, -0.12677482132145174, ...","[0.00051479205, 0.0008267846, 0.0007416491, 0....",0,0,"['死亡', '人数', '仍', '居高不下', '重', '特大事故', '尚未', '...","[1, 1]","[0.8987365365028381, 0.6969270706176758]","[0.00051479205, 0.0008267846, 0.0007416491, 0...."
2,12326,新华社 北京 月 日电 日本自民党 总务 会长 伊东 正义 先生 作为 日本首相 特使 预定...,"[0.3578358898089807, 0.08161251164610346, 0.11...","[0.0004722392, 0.00072869874, 0.00058751204, 0...",0,0,"['新华社', '北京', '月', '日电', '日本自民党', '总务', '会长', ...",[0],[0.7675070762634277],"[0.0004722392, 0.00072869874, 0.00058751204, 0..."
3,29706,本报 宁波 月 日电 记者 陆娅楠 月 日 杉杉 集团 有限公司 与 日本 伊藤忠 商事 株...,"[-0.303011174543276, 0.0032854388120253653, 0....","[0.00045990184, 0.0007084661, 0.0006132199, 0....",0,0,"['本报', '宁波', '月', '日电', '记者', '陆娅楠', '月', '日',...",[1],[0.6792555451393127],"[0.00045990184, 0.0007084661, 0.0006132199, 0...."
4,7697,新华社 一九七五年 一月 十五日 讯 据 日本 东方 通讯社 报道 一九七四年 日本 企业倒...,"[-0.0059179321907659906, -0.0731447363179498, ...","[0.0004908887, 0.0007480143, 0.000604546, 0.00...",0,0,"['新华社', '一九七五年', '一月', '十五日', '讯', '据', '日本', ...","[0, 0]","[0.8662652969360352, 0.9438633322715759]","[0.0004908887, 0.0007480143, 0.000604546, 0.00..."
5,2437,本报 东京 月 日电 记者 于青 报道 日本政府 内阁会议 今天 决定 向 中国 南部 水灾...,"[-0.02468244892408373, 0.16543349252372455, 0....","[0.00046329183, 0.00070480444, 0.0005240025, 0...",0,0,"['本报', '东京', '月', '日电', '记者', '于青', '报道', '日本政...",[1],[0.7693524360656738],"[0.00046329183, 0.00070480444, 0.0005240025, 0..."
6,13868,本报讯 记者 李丽辉 报道 日前 中国保监会 主席 吴定富 宣布 中国保监会 批准 英国 标...,"[0.056018661112242046, 0.027020653065623686, -...","[0.0004801159, 0.00074307574, 0.000648965, 0.0...",0,0,"['本报讯', '记者', '李丽辉', '报道', '日前', '中国保监会', '主席'...",[1],[0.7656781673431396],"[0.0004801159, 0.00074307574, 0.000648965, 0.0..."
7,18301,为期 两天 的 中国 东盟 文化 部长会议 与 东盟 中日韩 文化部长 系列 会议 于 日 ...,"[0.2538221689732428, -0.12466527668357397, -0....","[0.0004300173, 0.0006546416, 0.00044636868, 0....",0,0,"['为期', '两天', '的', '中国', '东盟', '文化', '部长会议', '与...",[1],[0.5380568504333496],"[0.0004300173, 0.0006546416, 0.00044636868, 0...."
8,6051,我 华侨 政策 不容 诋毁 越南 排华 是 为了 反华 苏越 合谋 反华 是因为 中国 坚决...,"[0.04909594366811509, -0.25454754978558125, -0...","[0.0005256694, 0.00087940495, 0.0010792044, 0....",0,0,"['我', '华侨', '政策', '不容', '诋毁', '越南', '排华', '是',...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.5756285190582275, 0.7522038817405701, 0.949...","[0.0005256694, 0.00087940495, 0.0010792044, 0...."
9,5333,从 二三百 美元 达到 一千美元 要 多少 年 问 编辑 同志 我们 想 了解 一下 世界 ...,"[0.005414972940424628, 0.006805516641449858, 0...","[0.00048636642, 0.00074270775, 0.0006064196, 0...",5,0,"['从', '二三百', '美元', '达到', '一千美元', '要', '多少', '年...","[1, 1, 1]","[0.5769215822219849, 0.8073752522468567, 0.929...","[0.00048636642, 0.00074270775, 0.0006064196, 0..."


In [12]:
df.to_csv("/content/drive/MyDrive/第三轮主动学习数据/0608_all_ls_score_token2.csv")